<a href="https://colab.research.google.com/github/sp8rks/MaterialsInformatics/blob/main/worked_examples/smiles_selfies/polymer_ml_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepChem and PubChemPy

DeepChem is an open-source library designed to faciliate machine learning for material science, chemistry, and biology. It has tools that can load and process datasets, easy to use featurizers, pre-built models, and evaluation metrics. PubChemPy is another free chemical database maintained by the National Center for Biotechnology Information (NCBI). In this notebook we will cover how to use each of the APIs for data and featurization as well as how to use them together. 

#### Video 

https://www.youtube.com/watch?v=kBk8HbjWwCw&list=PLL0SWcFqypCl4lrzk1dMWwTUrzQZFt7y0&index=15 (Molecular Strings and Fingerprints RDKit tutorial)

Google Colab link https://drive.google.com/file/d/1yTMpN7oWUXxqXqbiGEL-bh7f5mIrnZMj/view?usp=sharing

## Setup


### Imports

In [2]:
from tqdm import tqdm
from time import sleep
import numpy as np
import pandas as pd
import pubchempy as pcp
import deepchem as dc
import selfies as sf
from deepchem.feat import CircularFingerprint
from deepchem.metrics import Metric, mae_score

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (/opt/miniconda3/envs/masterENV/lib/python3.11/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/opt/miniconda3/envs/masterENV/lib/python3.11/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


## Loading DeepChem Datasets

One of the dataset groups that DeepChem works with is the MoleculeNet datasets. The DeepChem library is very handy in that some of the datasets are automatically split into a train/test/validation split. To load a dataset you will need to use the dc.molnet.load_X() function where X is the name of the dataset. 

In this notebook we will be using the ThermoSol dataset which contains thermodynamic solubility values for various compounds. The thermodynamic solubility of a compound indicates how much of the compound can dissolve given a specific temperature and pressure. 

The list of datasets available can be found here: https://deepchem.readthedocs.io/en/latest/api_reference/moleculenet.html


In [3]:
# Load the ThermoSol dataset with the default featurizer and splitting method
tasks, datasets, transformers = dc.molnet.load_thermosol()
train_dataset, val_dataset, test_dataset = datasets

n_tasks = len(tasks)
n_features = train_dataset.X.shape[1]

## About the Datasets

This dataset returns 3 things. Tasks, Data, and Transformers. What are they? 

### Tasks

Tasks are the specific prediction tasks or targets associated with the dataset. These tasks could be things like predicting solubility, toxicity, binding affinity, etc. For the HOPV dataset, the tasks are predicting properties related to organic photovoltaic materials such as electrochemical gap, optical gap, or the fill factor.

### Data

Datasets in DeepChem are objects that encapsulate the features, labels, and weights for the data. Often the different indexes of the dataset correspond to different splits. dataset[0] might be the training set, dataset[1] is the validation set, and so on. 

### Transformers

The transformers are used to preprocess the data before it is fed into the machine learning models. They can be used to normalize the data, perform feature scaling, handle missing values, etc. The transformers returned can be applied to the datasets to ensure they are more processed for training. 

Below we can see how many and what tasks there are and the size of the different datasets we have loaded.

In [4]:
print(f'Tasks: {tasks}')
print(f'Train dataset: {len(train_dataset)}')
print(f'Test dataset: {len(test_dataset)}')
print(f'Validation dataset: {len(val_dataset)}')

n_tasks = len(tasks)
print(f'Number of tasks: {n_tasks}')

Tasks: ['target']
Train dataset: 1410
Test dataset: 177
Validation dataset: 176
Number of tasks: 1


## PubChemPy

After retrieving the SMILES strings from the deepchem dataset we can pass it through PubChemPy to retrieve some information about the polymer. First we need to use PubChemPy to get the PubChem CID. We can do this through searching the database using the SMILES string. Once we have gotten the CID numbers we can use PubChemPy to get the other properties we want. For this we are finding the IUPAC name (the common english name), the molecular formula, and the molecular weight.

A handy short list of properties you can find with ChemPubPy are:
- cid
- elements 
- atoms
- bonds
- synonyms
- charge
- molecular_formula 
- molecular_weight
- canonical_smiles 
- isomeric_smiles 
- iupac_name
- exact_mass
- fingerprint
- heavy_atom_count
- isotope_atom_count
- atom_stereo_count 
- bond_stereo_count

The rest can be found in the documentation: https://pubchempy.readthedocs.io/en/latest/api.html

We will create a dataframe with 

In [50]:
polymerList = train_dataset.ids

df = pd.DataFrame(columns=['Molecular Formula', 'Molecular Weight', 'Heavy Atom Count', 'Atom Stereo Count'])

for polymer in polymerList:
    compound = pcp.get_compounds(polymer, 'smiles')
    molecularFormula = compound[0].molecular_formula
    molecularWeight = compound[0].molecular_weight
    heavyAtomCount = compound[0].heavy_atom_count
    atomStereoCount = compound[0].atom_stereo_count
    temp_df = pd.DataFrame({'Molecular Formula': [molecularFormula], 'Molecular Weight': [molecularWeight], 'Heavy Atom Count': [heavyAtomCount], 'Atom Stereo Count': [atomStereoCount]})
    df = pd.concat([df, temp_df], ignore_index=True)

KeyboardInterrupt: 

Alternatively we can also convert the SMILEs strings to SELFIEs strings. Selfie strings are pretty cool in that every SELFIE string is a valid molecule. Additionally, it has a lot more flexibility than SMILE strings in its representation of molecules. Generated SMILEs strings can end up being invalid molecules which can cause issues if they are being applied to machine learning applications. This is especially important when using generated molecular representations for Variational Autoencoders (VAEs) or Generative Adversarial Network (GANs).

PubChemPy has a built in method to convert to a SELFIE string from a SMILEs string. 

In [9]:
selfiesList = []
for polymer in polymerList:
    selfiesList.append(sf.encoder(polymer))

## PubChemPy Data

PubChemPy can also be used to search for specific compounds similar to the Materials Project Database. 

### Basic Usage

See https://pubchempy.readthedocs.io/en/latest/guide/gettingstarted.html#getting-started

The format of this query is that the first argument is the identifier and the second is the type of identifier. In the example below, the 'C6H10' is the query and the 'formula' tells the query that it is a formula. The types of queries include name, smiles, sdf, inchi, inchikey, and formula.

In [51]:
compound = pcp.get_compounds('C6H10', 'formula')

Once you have found a compound you can access it's properties. 

The properties that can be found are the same as the properties listed in the previous cell. It's important to index out the compound that you want. The query above generates a list of every single compound that matches your specification. 

In [52]:
print(compound[0].iupac_name)
print(compound[0].molecular_formula)
print(compound[0].molecular_weight)
print(compound[0].canonical_smiles)

cyclohexene
C6H10
82.14
C1CCC=CC1


To access all the properties from your query (and create a dataframe of them to work with later) you can use a for loop. 

In [53]:
df = pd.DataFrame(columns=['IUPAC Name', 'SMILES String', 'Molecular Formula', 'Molecular Weight'])
for compound in compound:
    smilesString = compound.canonical_smiles
    iupacName = compound.iupac_name
    molecularFormula = compound.molecular_formula
    molecularWeight = compound.molecular_weight
    temp_df = pd.DataFrame({'IUPAC Name': [iupacName], 'SMILES String': [smilesString], 'Molecular Formula': [molecularFormula], 'Molecular Weight': [molecularWeight]})
    df = pd.concat([df, temp_df], ignore_index=True)

In [54]:
df.head()
df.describe()

,IUPAC Name,SMILES String,Molecular Formula,Molecular Weight
count,240,285,285,285
unique,235,155,5,15
top,2-methylpent-1-ene,C1CCC=CC1,C6H10,82.14
freq,2,33,267,211


From this we can see that there was 235 unique compounds that matched this formula. Because we queried with a formula and not a smiles string there wasn't any specific connectivity that needed to occur. 

## DeepChem SMILES Featurization

DeepChem has some good built in featurizers. Here is a list of them: https://deepchem.readthedocs.io/en/latest/api_reference/featurizers.html#inorganic-crystal-featurizers

The one we will demonstrate is the OneHotFeaturizer. This is very similar to the OneHotEncoder from the CBFV notebook where it returns lists of numbers that represent our SMILEs string. 

In [59]:
# https://deepchem.readthedocs.io/en/latest/api_reference/featurizers.html#onehotfeaturizer
featurizer = dc.feat.OneHotFeaturizer()
smiles = df["SMILES String"]
encodings = featurizer.featurize(smiles)

# Print the type, shape, and untransformed representation of the first encoding
print(len(encodings))
print("type: ", type(encodings[0]))
print("shape: ", encodings[0].shape)
print("untransformed: ", featurizer.untransform(encodings[0]))

# Print the first encoding
print(encodings[0])
print(encodings[0].sum())

285
type:  <class 'numpy.ndarray'>
shape:  (100, 35)
untransformed:  C1CCC=CC1
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
100.0


As we can see here each encoding is 100x35. There are 285 encodings which matches our number of datapoints that we had before. Something else that's really cool is that the DeepChem featurizer is able to untransform our encodings back to SMILEs strings!

## SELFIES Featurization



### One-hot Encoding Example

SELFIEs can also be featurized. This is very helpful for turning SELFIE strings into numerical inputs for machine learning models. You have to first have a defined list of SELFIEs strings. This can either be an explicitly defined list as shown below or a SELFIEs list created from a list of SMILEs. Next you extract the alphabet (the unique symbols) used in the list. Next you calculate the padding which is found from the maximum SELFIE string length. This makes sure that all the encoded sequences are the same size. Next you map each alphabet symbol to a unique integer index. The sf.selfies_to_encoding function then encodes all the SELFIE strings.

In [ ]:
# https://github.com/aspuru-guzik-group/selfies#integer-and-one-hot-encoding-selfies
dataset = ["[C][O][C]", "[F][C][F]", "[O][=O]", "[C][C][O][C][C]"]
alphabet = sf.get_alphabet_from_selfies(dataset)
alphabet.add("[nop]")  # [nop] is a special padding symbol
alphabet = list(sorted(alphabet))  # ['[=O]', '[C]', '[F]', '[O]', '[nop]']

pad_to_len = max(sf.len_selfies(s) for s in dataset)  # 5
symbol_to_idx = {s: i for i, s in enumerate(alphabet)}

dimethyl_ether = dataset[0]  # [C][O][C]

label, one_hot = sf.selfies_to_encoding(
   selfies=dimethyl_ether,
   vocab_stoi=symbol_to_idx,
   pad_to_len=pad_to_len,
   enc_type="both"
)
print(label, one_hot)

[1, 3, 1, 4, 4] [[0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1]]


## Try It Yourself!

- Using DeepChem load the thermosol dataset (use the documentation)
- Using PubChemPy extract the molecular weight, atom stereo count, covalent unit count, and the isotope atom count and throw the DeepChem SMILEs strings, PubChemPy properties, and the target values into a dataframe
- Next, featurize the SMILEs string with a OneHotFeaturizer
- Lastly, split the data into train/test/validation splits and train a random forest model on the data. Print the mean accuracy error score.